In [2]:
import pandas as pd
import os

flat = pd.read_excel('flat_final.xlsx')

records = os.listdir('records')
del (records[0])

rsn = flat['Record Sequence Number'].to_list()
file_x = flat['File Name (Horizontal 1)'].to_list()
file_y = flat['File Name (Horizontal 2)'].to_list()

In [3]:
def ReadPeer_acc(filepath):

    fileName = filepath.split(os.path.sep)[-1].replace(".at2", "")
    with open(filepath) as f:
        acc = []
        for no, line in enumerate(f.readlines()):
            if no == 3:
                no_data = int(line.split(",")[0].split()[1])
                dt = float(line.split(",")[1].split()[1])
            if no > 3:
                [acc.append(float(item)) for item in line.split()]
    time = [dt * item for item in range(no_data)]
    return(acc)

In [5]:
def ReadPeer_dt(filepath):

    fileName = filepath.split(os.path.sep)[-1].replace(".at2", "")
    with open(filepath) as f:
        acc = []
        for no, line in enumerate(f.readlines()):
            if no == 3:
                no_data = int(line.split(",")[0].split()[1])
                dt = float(line.split(",")[1].split()[1])
            if no > 3:
                [acc.append(float(item)) for item in line.split()]
    time = [dt * item for item in range(no_data)]
    return(dt)

In [6]:
def ResponseSpectra(acceleration, sampling_interval, damping_ratio = 0.05, Tmax = 4, T_int = 0.05):

    '''      
    Response spectra using piecewise
    
    Input:
        T: vector with periods (s)
        s: acceleration time series
        zi: damping ratio
        dt: time steps for s
    
    Returns:
        PSA, PSV, SA, SV, SD
    '''

    import numpy as np
    T = np.arange(  T_int , Tmax + T_int , T_int)
    s = acceleration
    zi = damping_ratio
    dt = sampling_interval

    pi = np.pi
    
    nper = np.size(T)						      # number of natural periods
    n    = np.size(s)                             # length of record
    
    SD   = np.zeros(nper)				          # rel. displac. spectrum
    SV   = np.zeros(nper)				          # rel. vel. spectrum
    SA   = np.zeros(nper)				          # total acc. spectrum	
     
    
    for k in range(nper):
       wn = 2*pi/T[k]
       wd = wn*(1-zi**2)**(1/2)
       
       u = np.zeros((2,n))          # matrix with velocities and displacements
       
       ex = np.exp(-zi*wn*dt)
       cwd = np.cos(wd*dt)
       swd = np.sin(wd*dt)
       zisq = 1/(np.sqrt(1-(zi**2)))
    
       a11 = ex*(cwd+zi*zisq*swd)
       a12 = (ex/wd)*swd
       a21 = -wn*zisq*ex*swd
       a22 = ex*(cwd-zi*zisq*swd)
    
       b11 = ex*(((2*zi**2-1)/((wn**2)*dt)+zi/wn)*(1/wd)*np.sin(wd*dt)+
           (2*zi/((wn**3)*dt)+1/(wn**2))*np.cos(wd*dt))-2*zi/((wn**3)*dt)
       b12 = -ex*(((2*zi**2-1)/((wn**2)*dt))*(1/wd)*np.sin(wd*dt)+
           (2*zi/((wn**3)*dt))*np.cos(wd*dt))-(1/(wn**2))+2*zi/((wn**3)*dt)
       b21 = -((a11-1)/((wn**2)*dt))-a12
       b22 = -b21-a12
       
       A = np.array([[a11,a12],[a21,a22]])
       B = np.array([[b11,b12],[b21,b22]])
    
       for q in range(n-1):
          u[:,q+1] = np.dot(A,u[:,q]) + np.dot(B,np.array([s[q],s[q+1]]))
       
       at = -2*wn*zi*u[1,:]-(wn**2)*u[0,:]
       
       SD[k]   = np.max( np.abs(u[0,:]) )
       SV[k]   = np.max( np.abs(u[1,:]) )
       SA[k]   = np.max( np.abs(at) )
    
    PSV = (2*pi/T)*SD                    # pseudo-vel. spectrum
    PSA = (2*pi/T)**2 *SD  	             # pseudo-accel. spectrum
    
    return SA

In [7]:
spectra_x = {}
spectra_y = {}
error_list_x = []
error_list_y = []

for i, j in zip(rsn, file_x):
    try:
        spectra_x[i] = ResponseSpectra(ReadPeer_acc('records/' + j), ReadPeer_dt('records/' + j))
        print('okay in x ' + j)
    except:
        error_list_x.append(j)
records_spectra_x = pd.DataFrame(spectra_x)
records_spectra_x.to_csv('records_spectra_x.csv')

for i, j in zip(rsn, file_y):
    try:
        spectra_y[i] = ResponseSpectra(ReadPeer_acc('records/' + j), ReadPeer_dt('records/' + j))
        print('okay in y ' + j)
    except:
        error_list_y.append(j)
records_spectra_y = pd.DataFrame(spectra_y)
records_spectra_y.to_csv('records_spectra_y.csv')

okay in x RSN1_HELENA.A_A-HMC180.AT2
okay in x RSN2_AAD_B-FEB000.AT2
okay in x RSN3_AAD_FRN225.AT2
okay in x RSN4_IMPVALL.BG_B-ELC000.AT2
okay in x RSN5_AAD_A-FRN045.AT2
okay in x RSN6_IMPVALL.I_I-ELC180.AT2
okay in x RSN7_AAD_C-FRN045.AT2
okay in x RSN8_AAD_F-FRN225.AT2
okay in x RSN10_AAD_C-ELC000.AT2
okay in x RSN13_AAD_PAS180.AT2
okay in x RSN14_AAD_SBA042.AT2
okay in x RSN15_AAD_TAF021.AT2
okay in x RSN16_AAD_A-FRN044.AT2
okay in x RSN18_AAD_G-ELC000.AT2
okay in x RSN20_NCALIF.FH_H-FRN044.AT2
okay in x RSN21_AAD_E-ELC000.AT2
okay in x RSN22_ELALAMO_ELC180.AT2
okay in x RSN23_AAD_GGP010.AT2
okay in x RSN27_AAD_C-HCH181.AT2
okay in x RSN28_PARKF_C12050.AT2
okay in x RSN30_PARKF_C05085.AT2
okay in x RSN31_PARKF_C08050.AT2
okay in x RSN33_AAD_TMB205.AT2
okay in x RSN34_AAD_C-FRN224.AT2
okay in x RSN35_AAD_E-HCH181.AT2
okay in x RSN36_AAD_A-ELC180.AT2
okay in x RSN37_AAD_A-PEL090.AT2
okay in x RSN38_AAD_A-TLI249.AT2
okay in x RSN40_AAD_A-SON033.AT2
okay in x RSN45_AAD_DCF090.AT2
okay i

In [1]:
records_spectra_x = pd.DataFrame(spectra_x)
records_spectra_y = pd.DataFrame(spectra_y)

NameError: name 'pd' is not defined

In [17]:
records_spectra_x.to_csv('records_spectra_x.csv')
records_spectra_y.to_csv('records_spectra_y.csv')

In [1]:
spectra_x

NameError: name 'spectra_x' is not defined